<a href="https://colab.research.google.com/github/yusuf-yesil/RAG_project/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install chromadb --quiet
%pip install sentence_transformers --quiet
%pip install langchain --quiet
%pip install tqdm --quiet

import langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import SentenceTransformersTokenTextSplitter


from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from chromadb import Client, PersistentClient
from chromadb.utils import embedding_functions

import textwrap
from IPython.display import display
from IPython.display import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [19]:
def create_chroma_client(collection_name, embedding_function):
  chroma_client = Client()
  chroma_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=embedding_function)
  return chroma_client, chroma_collection

In [20]:
import tqdm

In [18]:
collection_name = "Lines"
sentence_transformer_model="sentence-transformers/multi-qa-distilbert-cos-v1" #"distiluse-base-multilingual-cased-v1"
embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=sentence_transformer_model)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
chroma_client, chroma_collection = create_chroma_client(collection_name,
                                                        embedding_function)

In [22]:
# Verify collection properties
print(f"Collection name: {chroma_collection.name}")
print(f"Number of documents in collection: {chroma_collection.count()}")

# List all collections in the client
print("All collections in ChromaDB client:")
for collection in chroma_client.list_collections():
    print(collection.name)

Collection name: Lines
Number of documents in collection: 0
All collections in ChromaDB client:
Lines


In [23]:
from google.colab import files
def upload_multiple_files():
  uploaded = files.upload()
  file_names = list()
  for fn in uploaded.keys():
    #print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
    file_names.append(fn)
  return file_names

In [24]:
import pandas as pd
from textwrap import wrap


In [44]:
def extract_log_rows(csv_path):
    # CSV dosyasını oku
    log_data = pd.read_csv(csv_path)
    # Her bir log satırını işle ve listeye ekle
    log_rows = []
    for index, row in log_data.iterrows():
        # Log satırını birleştir
        log_row = f"IP: {row['IP Address']}, Timestamp: {row['Timestamp']}, Request: {row['Request']}, Status Code: {row['Status Code']}, Response Size: {row['Response Size']}, Referer: {row['Referer']}, User Agent: {row['User Agent']}"
        log_rows.append(log_row)
    return log_rows


In [45]:
def chunk_log_data(log_rows, chunk_size=2000):
    """
    Log satırlarını belirli büyüklüklerdeki parçalara böler.

    Parameters:
    - log_rows: List of log rows as strings.
    - chunk_size: Maksimum karakter sayısı olan chunk büyüklüğü.

    Returns:
    - List of chunks with the specified maximum character size.
    """
    # Birleştirilmiş log metnini oluştur
    combined_text = "\n".join(log_rows)

    # Karakter limitine göre parçaları oluştur
    chunks = wrap(combined_text, chunk_size)

    return chunks


In [46]:
def convert_Chunk_Token(text_chunksinChar, sentence_transformer_model, chunk_overlap=0, tokens_per_chunk=256):
    token_splitter = SentenceTransformersTokenTextSplitter(
        chunk_overlap=chunk_overlap,
        model_name=sentence_transformer_model,
        tokens_per_chunk=tokens_per_chunk
    )

    text_chunksinTokens = []
    for text in text_chunksinChar:
        text_chunksinTokens += token_splitter.split_text(text)

    print(f"\nTotal number of chunks (document split by {tokens_per_chunk} tokens per chunk): {len(text_chunksinTokens)}")
    return text_chunksinTokens

In [47]:
def add_meta_data(log_chunksinTokens, title, category, initial_id):
  ids = [str(i+initial_id) for i in range(len(log_chunksinTokens))]
  metadata = {
      'document': title,
      'category': category
  }
  metadatas = [ metadata for i in range(len(log_chunksinTokens))]
  return ids, metadatas

In [48]:
"""def add_document_to_collection(ids, metadatas, log_chunksinTokens, chroma_collection):
  print("Before inserting, the size of the collection: ", chroma_collection.count())
  chroma_collection.add(ids=ids, metadatas= metadatas, documents=log_chunksinTokens)
  print("After inserting, the size of the collection: ", chroma_collection.count())
  return chroma_collection"""

'def add_document_to_collection(ids, metadatas, log_chunksinTokens, chroma_collection):\n  print("Before inserting, the size of the collection: ", chroma_collection.count())\n  chroma_collection.add(ids=ids, metadatas= metadatas, documents=log_chunksinTokens)\n  print("After inserting, the size of the collection: ", chroma_collection.count())\n  return chroma_collection'

In [49]:
def add_document_to_collection(ids, metadatas, log_chunksinTokens, chroma_collection):
    try:
        # Mevcut koleksiyon boyutunu yazdır
        print("Before inserting, the size of the collection: ", chroma_collection.count())
        # Veri ekleme işlemini gerçekleştir
        chroma_collection.add(ids=ids, metadatas=metadatas, documents=log_chunksinTokens)
        print(ids[10],metadatas[10],log_chunksinTokens[10])
        # Ekleme işlemi sonrasında koleksiyon boyutunu yazdır
        print("After inserting, the size of the collection: ", chroma_collection.count())
    except Exception as e:
        # Hata oluşursa bu blok çalışır
        print(f"An error occurred while adding documents to the collection: {str(e)}")
    return chroma_collection

In [32]:
def retrieveDocs(chroma_collection, query, n_results=5, return_only_docs=False):
    results = chroma_collection.query(query_texts=[query],
                                      include= [ "documents","metadatas",'distances' ],
                                      n_results=n_results)

    if return_only_docs:
        return results['documents'][0]
    else:
        return results

In [33]:
def show_results(results, return_only_docs=False):

  if return_only_docs:
    retrieved_documents = results
    if len(retrieved_documents) == 0:
      print("No results found.")
      return
    for i, doc in enumerate(retrieved_documents):
      print(f"Document {i+1}:")
      print("\tDocument Text: ")
      display(to_markdown(doc));
  else:

      retrieved_documents = results['documents'][0]
      if len(retrieved_documents) == 0:
          print("No results found.")
          return
      retrieved_documents_metadata = results['metadatas'][0]
      retrieved_documents_distances = results['distances'][0]
      print("------- retreived documents -------\n")

      for i, doc in enumerate(retrieved_documents):
          print(f"Document {i+1}:")
          print("\tDocument Text: ")
          display(to_markdown(doc));
          print(f"\tDocument Source: {retrieved_documents_metadata[i]['document']}")
          print(f"\tDocument Source Type: {retrieved_documents_metadata[i]['category']}")
          print(f"\tDocument Distance: {retrieved_documents_distances[i]}")


In [50]:
def load_multiple_csvs_to_ChromaDB(collection_name,sentence_transformer_model):

  collection_name= collection_name
  category= "Journal Paper"
  sentence_transformer_model=sentence_transformer_model
  embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction(model_name=sentence_transformer_model)
  chroma_client, chroma_collection = create_chroma_client(collection_name, embedding_function)
  current_id = chroma_collection.count()
  file_names = upload_multiple_files()
  for file_name in file_names:
    print(f"Document: {file_name} is being processed to be added to the {chroma_collection.name} {chroma_collection.count()}")
    print(f"current_id: {current_id} ")
    #pdf_texts = convert_PDF_Text(file_name)
    #text_chunksinChar = convert_Page_ChunkinChar(pdf_texts)
    log_rows = extract_log_rows(file_name)
    print(f"log satırları:{len(log_rows)}")
    print(f"2.nci log satırı:{log_rows[2]}")
    for i in range(10):
        print(log_rows[i+1])
    #chunks = chunk_log_data(log_rows)
    #print(f"chunk sayısı:{len(chunks)}")
    tokens = convert_Chunk_Token(log_rows, sentence_transformer_model)
    #text_chunksinTokens = tokenize_log_data(chunk_logs,sentence_transformer_model)
    ids,metadatas = add_meta_data(tokens,file_name,category, current_id)
    current_id = current_id + len(tokens)
    chroma_collection = add_document_to_collection(ids, metadatas, tokens, chroma_collection)
    print(f"Document: {file_name} added to the collection: {chroma_collection.count()}")
  return  chroma_client, chroma_collection

In [51]:
chroma_client, chroma_collection= load_multiple_csvs_to_ChromaDB(collection_name,sentence_transformer_model)

Saving duzenli_log_dosyasi_400.csv to duzenli_log_dosyasi_400.csv
Document: duzenli_log_dosyasi_400.csv is being processed to be added to the Lines 0
current_id: 0 
log satırları:380
2.nci log satırı:IP: 14.49.42.25, Timestamp: 12/May/2022:01:24:06 +0000, Request: GET /articles/dynamic-dns-with-dhcp/ HTTP/1.1, Status Code: 200, Response Size: 18848, Referer: -, User Agent: Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5
IP: 14.49.42.25, Timestamp: 12/May/2022:01:24:15 +0000, Request: GET /articles/openldap-with-saslauthd/ HTTP/1.1, Status Code: 200, Response Size: 12700, Referer: -, User Agent: Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5
IP: 14.49.42.25, Timestamp: 12/May/2022:01:24:06 +0000, Request: GET /articles/dynamic-dns-with-dhcp/ HTTP/1.1, Status Code: 200, Response Size: 18848, Referer: -, User Agent: Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox

In [52]:
query = "14.49.42.25,12/May/2022:01:24:44 +0000,GET /articles"

'''
14.49.42.25,12/May/2022:01:24:44 +0000,GET /articles/ppp-over-ssh/ HTTP/1.1,200,18586,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5

'''

'\n14.49.42.25,12/May/2022:01:24:44 +0000,GET /articles/ppp-over-ssh/ HTTP/1.1,200,18586,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5\n\n'

In [54]:
retrieved_documents=retrieveDocs(chroma_collection, query, 10)
show_results(retrieved_documents)

------- retreived documents -------

Document 1:
	Document Text: 


> ip : 14. 49. 42. 25, timestamp : 12 / may / 2022 : 01 : 25 : 25 + 0000, request : get / articles / week - of - unix - tools / http / 1. 1, status code : 200, response size : 9313, referer : -, user agent : mozilla / 5. 0 ( windows ; u ; windows nt 6. 1 ; en - us ; rv : 1. 9. 2b1 ) gecko / 20091014 firefox / 3. 6b1 gtb5

	Document Source: duzenli_log_dosyasi_400.csv
	Document Source Type: Journal Paper
	Document Distance: 0.9381861686706543
Document 2:
	Document Text: 


> ip : 31. 133. 11. 222, timestamp : 11 / may / 2022 : 21 : 55 : 21 + 0000, request : get / blog / articles / week - of - unix - tools / day - 3 - awk. html http / 1. 0, status code : 200, response size : 15457, referer : http : / / www. semicomplete. com / blog / articles / week - of - unix - tools / day - 3 - awk. html, user agent : mozilla / 5. 0 ( windows nt 6. 1 ; wow64 ) applewebkit / 536. 5 ( khtml, like gecko ) yabrowser / 1. 1. 1084. 5409 chrome / 19. 1. 1084. 5409 safari / 536. 5

	Document Source: duzenli_log_dosyasi_400.csv
	Document Source Type: Journal Paper
	Document Distance: 0.9581603407859802
Document 3:
	Document Text: 


...

	Document Source: duzenli_log_dosyasi_400.csv
	Document Source Type: Journal Paper
	Document Distance: 1.141883373260498
Document 9:
	Document Text: 


> ip : 61. 55. 141. 10, timestamp : 12 / may / 2022 : 01 : 26 : 32 + 0000, request : get / blog / tags / ideas http / 1. 0, status code : 200, response size : 30011, referer : -, user agent : mozilla / 5. 0 ( windows ; u ; windows nt 6. 1 ; en - us ; rv : 1. 9. 2b1 ) gecko / 20091014 firefox / 3. 6b1 gtb5

	Document Source: duzenli_log_dosyasi_400.csv
	Document Source Type: Journal Paper
	Document Distance: 1.1420154571533203
Document 10:
	Document Text: 


> ip : 78. 230. 246. 16, timestamp : 12 / may / 2022 : 01 : 50 : 38 + 0000, request : get / presentations / logstash - monitorama - 2013 / css / theme / night - jls. css http / 1. 1, status code : 200, response size : 3811, referer : http : / / semicomplete. com / presentations / logstash - monitorama - 2013 /, user agent : mozilla / 5. 0 ( x11 ; linux x86 _ 64 ; rv : 17. 0 ) gecko / 20100101 firefox / 17. 0

	Document Source: duzenli_log_dosyasi_400.csv
	Document Source Type: Journal Paper
	Document Distance: 1.147687554359436


In [58]:
chroma_collection.get(ids=['218'])

{'ids': ['218'],
 'embeddings': None,
 'metadatas': [{'category': 'Journal Paper',
   'document': 'duzenli_log_dosyasi_400.csv'}],
 'documents': ['ip : 5. 10. 83. 65, timestamp : 11 / may / 2022 : 22 : 21 : 57 + 0000, request : get / blog / tags / code http / 1. 1, status code : 200, response size : 14472, referer : -, user agent : mozilla / 5. 0 ( compatible ; ahrefsbot / 5. 0 ; + http : / / ahrefs. com / robot / )'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [61]:
query = "IP:\"14.49.42.25\""

In [63]:
query="Request : GET / blog / geekery / bypassing - captive - portals. html HTTP / 1. 1 nerdedir"

In [64]:
retrieved_documents=retrieveDocs(chroma_collection, query, 10)
show_results(retrieved_documents)

------- retreived documents -------

Document 1:
	Document Text: 


> ip : 161. 162. 84. 82, timestamp : 11 / may / 2022 : 21 : 44 : 40 + 0000, request : get / blog / geekery / bypassing - captive - portals. html http / 1. 1, status code : 200, response size : 10613, referer : http : / / www. google. com / url? sa = t & rct = j & q = & esrc = s & frm = 1 & source = web & cd = 1 & ved = 0ccuqfjaa & url = http % 3a % 2f % 2fwww. semicomplete. com % 2fblog % 2fgeekery % 2fbypassing - captive - portals. html & ei = 47weu5immdsmkaee3yhqbg & usg = afqjcnhqab9hldpowymootqhefhojiv0jg & bvm = bv. 61535280, d. ew0, user agent : mozilla / 4. 0 ( compatible ; msie 8. 0 ; windows nt 6. 1 ; wow64

	Document Source: duzenli_log_dosyasi_400.csv
	Document Source Type: Journal Paper
	Document Distance: 0.4273245334625244
Document 2:
	Document Text: 


> ip : 161. 162. 84. 82, timestamp : 11 / may / 2022 : 21 : 44 : 40 + 0000, request : get / images / jordan - 80. png http / 1. 1, status code : 200, response size : 6146, referer : http : / / www. semicomplete. com / blog / geekery / bypassing - captive - portals. html, user agent : mozilla / 4. 0 ( compatible ; msie 8. 0 ; windows nt 6. 1 ; wow64 ; trident / 4. 0 ; slcc2 ;. net clr 2. 0. 50727 ;. net clr 3. 5. 30729 ;. net clr 3. 0. 30729 ; media center pc 6. 0 ;. net4. 0c ; infopath. 3 ; ms - rtc lm 8 ;. net4. 0e )

	Document Source: duzenli_log_dosyasi_400.csv
	Document Source Type: Journal Paper
	Document Distance: 0.5681900978088379
Document 3:
	Document Text: 


...

	Document Source: duzenli_log_dosyasi_400.csv
	Document Source Type: Journal Paper
	Document Distance: 1.0736541748046875
Document 9:
	Document Text: 


> ip : 70. 194. 143. 83, timestamp : 11 / may / 2022 : 22 : 21 : 49 + 0000, request : get / http / 1. 1, status code : 200, response size : 37932, referer : -, user agent : reeder / 2. 1. 1 cfnetwork / 672. 0. 8 darwin / 14. 0. 0

	Document Source: duzenli_log_dosyasi_400.csv
	Document Source Type: Journal Paper
	Document Distance: 1.0771918296813965
Document 10:
	Document Text: 


> ip : 218. 30. 103. 62, timestamp : 11 / may / 2022 : 22 : 15 : 38 + 0000, request : get / blog / geekery / comcast - dns - hijack - breaks - things. html http / 1. 1, status code : 200, response size : 18095, referer : -, user agent : sogou web spider / 4. 0 ( + http : / / www. sogou. com / docs / help / webmasters. htm # 07 )

	Document Source: duzenli_log_dosyasi_400.csv
	Document Source Type: Journal Paper
	Document Distance: 1.0772368907928467


In [65]:
!pip install -q -U google-generativeai

In [66]:
import os
import textwrap
import google.generativeai as genai
from IPython.display import display
from IPython.display import Markdown

In [67]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [68]:
def build_chatBot(system_instruction):
  model = genai.GenerativeModel('gemini-1.5-flash-latest', system_instruction=system_instruction)
  chat = model.start_chat(history=[])
  return chat

In [69]:
def generate_LLM_answer(prompt, context, chat):
  response = chat.send_message( prompt + context)
  return response.text

In [70]:
# Used to securely store your API key
from google.colab import userdata
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [71]:
system_prompt= """
You are an attentive and supportive system assistant.

Your role is to provide assistance based solely on the provided context.

Here’s how we’ll proceed:

I will provide you with a log file and related text excerpt.
Your task is to answer questions based only on the provided partial texts.
If the answer isn’t explicitly found within the given context, respond with 'I don't know.'
After each response, provide a detailed explanation. Break down your answer step by step and relate it directly to the provided context.
Sometimes, I will ask questions about the chat session, such as summarizing the chat or listing questions. For these types of questions, avoid using the provided partial texts.
Generate the answer in the same language as the given question.
If you are ready, I will provide you with the question and context.
"""

In [72]:
RAG_LLM = build_chatBot(system_prompt)

In [81]:
prompt="Bu IP adresi geçen satırları göster: 14.49.42.25"
context= """IP Address,Timestamp,Request,Status Code,Response Size,Referer,User Agent
14.49.42.25,12/May/2022:01:24:44 +0000,GET /articles/ppp-over-ssh/ HTTP/1.1,200,18586,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5
14.49.42.25,12/May/2022:01:24:15 +0000,GET /articles/openldap-with-saslauthd/ HTTP/1.1,200,12700,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5
14.49.42.25,12/May/2022:01:24:06 +0000,GET /articles/dynamic-dns-with-dhcp/ HTTP/1.1,200,18848,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5

"""

In [86]:
response=generate_LLM_answer(prompt, context, RAG_LLM)
to_markdown(response)

> The log file shows three lines with the IP address 14.49.42.25.  
> 
> Here they are: 
> 
> * 14.49.42.25,12/May/2022:01:24:44 +0000,GET /articles/ppp-over-ssh/ HTTP/1.1,200,18586,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5
> * 14.49.42.25,12/May/2022:01:24:15 +0000,GET /articles/openldap-with-saslauthd/ HTTP/1.1,200,12700,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5
> * 14.49.42.25,12/May/2022:01:24:06 +0000,GET /articles/dynamic-dns-with-dhcp/ HTTP/1.1,200,18848,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5 
> 
> The log file contains details about requests made to a server. Each line represents a single request.  The IP address, timestamp, request, status code, response size, referer, and user agent are listed for each request. 
> 
> I identified the lines based on the IP address '14.49.42.25' found at the beginning of each line in the log file.


In [87]:
RAG_LLM.history

[parts {
   text: "Bu IP adresi ge\303\247en sat\304\261rlar\304\261 g\303\266ster: 14.49.42.25IP Address,Timestamp,Request,Status Code,Response Size,Referer,User Agent\n14.49.42.25,12/May/2022:01:24:44 +0000,GET /articles/ppp-over-ssh/ HTTP/1.1,200,18586,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5\n14.49.42.25,12/May/2022:01:24:15 +0000,GET /articles/openldap-with-saslauthd/ HTTP/1.1,200,12700,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5\n14.49.42.25,12/May/2022:01:24:06 +0000,GET /articles/dynamic-dns-with-dhcp/ HTTP/1.1,200,18848,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5\n\n"
 }
 role: "user",
 parts {
   text: "The log file shows three lines with the IP address 14.49.42.25.  \n\nHere they are: \n\n* 14.49.42.25,12/May/2022:01:24:44 +0000,GET /articles/ppp-over-ssh/ HTTP/1.1,200,18586,-,Mozilla/5.0 (Windows; U; Windows NT 6.1; e

In [88]:
RAG_LLM.history.clear()
RAG_LLM.history

[]

In [89]:
# Verify collection properties
print(f"Collection name: {chroma_collection.name}")  # Access the name attribute directly
print(f"Number of documents in collection: {chroma_collection.count()}")

# List all collections in the client
print("All collections in ChromaDB client:")
for collection in chroma_client.list_collections():
    print(collection.name)

Collection name: Lines
Number of documents in collection: 390
All collections in ChromaDB client:
Lines


In [90]:
def summarize_collection(chroma_collection):
  summary = [] # Initialize summary as a list
  print("Summarizing the collection...")
  # Verify collection properties
  print(f"\t Collection name: {chroma_collection.name}")  # Access the name attribute directly
  print(f"\t Number of document chunks in collection: {chroma_collection.count()}")
  summary.append(f"Collection name: {chroma_collection.name}") # Append to the list
  summary.append(f"Number of document chunks in collection: {chroma_collection.count()}")
  # Print distinct metadata "document" for each chunk in the collection
  print("\t Distinct 'document' metadata in the collection:")
  distinct_documents = set()  # Use a set to store unique document names

  # Iterate over chunks in the collection
  for chunk_id in range(chroma_collection.count()):
      metadata = chroma_collection.get([str(chunk_id)])['metadatas'][0]  # Get metadata for the chunk
      document_name = metadata.get("document", "Unknown")  # Get document metadata; default to "Unknown" if not present
      distinct_documents.add(document_name)  # Add document name to set for uniqueness

  # Print all distinct document names
  summary.append("Documents:")
  for document_name in distinct_documents:
      print("\t ",document_name)
      summary.append(document_name) # Append to the list

  print("Collection summarization completed.")

  # Join the list elements into a single string
  summary_string = "\n ".join(summary)
  return summary_string

In [91]:
s=summarize_collection(chroma_collection)

Summarizing the collection...
	 Collection name: Lines
	 Number of document chunks in collection: 390
	 Distinct 'document' metadata in the collection:
	  duzenli_log_dosyasi_400.csv
Collection summarization completed.


In [92]:
print(s)

Collection name: Lines
 Number of document chunks in collection: 390
 Documents:
 duzenli_log_dosyasi_400.csv


In [93]:
def generateAnswer(RAG_LLM, chroma_collection,query,n_results=5, only_response=True):
    retrieved_documents= retrieveDocs(chroma_collection, query, 10, return_only_docs=True)
    prompt = "QUESTION: "+ query
    context = "\n EXCERPTS: "+ "\n".join(retrieved_documents)
    if not only_response:
      print("------- retreived documents -------\n")
      for i, doc in enumerate(retrieved_documents):
        print(f"Document {i+1}:")
        print(f"\tDocument Text: {doc}")
      print("------- RAG answer -------\n")
    output = generate_LLM_answer(prompt, context, RAG_LLM)

    display(to_markdown(output))
    print('\n')
    return output

In [94]:
queries =["Which URLs were accessed in the last 5 minutes?",
          "What was the last request and what was the response status?",
          "Which pages were last accessed by the user with IP address 14.49.42.25?",
          "Which URLs were visited by the user with the User Agent 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1)'?",
          "What are the most common HTTP status codes?",
          "What are the 5 most recent requests with HTTP status code 200?",
          "Which referer URLs were visited the most?",
          "What are the requests with a specific referrer URL?",
          "What are all requests between 01:24:00 and 01:25:00 on May 12, 2022?",
          "Which request had the longest response time?"
    ]

In [96]:
reply=generateAnswer(RAG_LLM, chroma_collection, queries[5],10, only_response=False)

------- retreived documents -------

Document 1:
	Document Text: ip : 188. 40. 60. 225, timestamp : 11 / may / 2022 : 21 : 43 : 11 + 0000, request : get / http / 1. 1, status code : 200, response size : 37932, referer : -, user agent : mozilla / 5. 0 ( windows ; u ; windows nt 5. 1 ; en ; rv : 1. 9. 1. 3 ) gecko / 20090824 firefox / 3. 5. 3
Document 2:
	Document Text: ip : 14. 49. 42. 25, timestamp : 12 / may / 2022 : 01 : 25 : 25 + 0000, request : get / articles / week - of - unix - tools / http / 1. 1, status code : 200, response size : 9313, referer : -, user agent : mozilla / 5. 0 ( windows ; u ; windows nt 6. 1 ; en - us ; rv : 1. 9. 2b1 ) gecko / 20091014 firefox / 3. 6b1 gtb5
Document 3:
	Document Text: ip : 63. 223. 125. 246, timestamp : 11 / may / 2022 : 22 : 15 : 11 + 0000, request : get / http / 1. 0, status code : 200, response size : 37932, referer : http : / / www. semicomplete. com /, user agent : mozilla / 5. 0 ( macintosh ; intel mac os x 10. 8 ; rv : 22. 0 ) gecko / 2

> The 5 most recent requests with HTTP status code 200 are:
> 
> 1. **ip : 189. 90. 160. 60, timestamp : 12 / may / 2022 : 01 : 40 : 08 + 0000, request : get / reset. css http / 1. 0, status code : 200, response size : 1015, referer : http : / / www. semicomplete. com / projects / xdotool /, user agent : mozilla / 5. 0 ( x11 ; linux i686 on x86 _ 64 ; rv : 27. 0 ) gecko / 20100101 firefox / 27. 0**
> 2. **ip : 189. 90. 160. 60, timestamp : 12 / may / 2022 : 01 : 40 : 09 + 0000, request : get / images / web / 2009 / banner. png http / 1. 0, status code : 200, response size : 52315, referer : http : / / www. semicomplete. com / style2. css, user agent : mozilla / 5. 0 ( x11 ; linux i686 on x86 _ 64 ; rv : 27. 0 ) gecko / 20100101 firefox / 27. 0** 
> 3. **ip : 190. 228. 21. 2, timestamp : 12 / may / 2022 : 01 : 36 : 41 + 0000, request : get / images / web / 2009 / banner. png http / 1. 1, status code : 200, response size : 52315, referer : http : / / www. semicomplete. com / style2. css, user agent : mozilla / 5. 0 ( x11 ; linux x86 _ 64 ; rv : 18. 0 ) gecko / 20100101 firefox / 18. 0 iceweasel / 18. 0. 2** 
> 4. **ip : 137. 108. 145. 40, timestamp : 12 / may / 2022 : 01 : 28 : 54 + 0000, request : get /? page = 2 http / 1. 1, status code : 200, response size : 36673, referer : -, user agent : mozilla / 5. 0 ( windows nt 6. 1 ) applewebkit / 537. 36 ( khtml, like gecko ) chrome / 32. 0. 1700. 102 safari / 537. 36**
> 5. **ip : 14. 49. 42. 25, timestamp : 12 / may / 2022 : 01 : 25 : 25 + 0000, request : get / articles / week - of - unix - tools / http / 1. 1, status code : 200, response size : 9313, referer : -, user agent : mozilla / 5. 0 ( windows ; u ; windows nt 6. 1 ; en - us ; rv : 1. 9. 2b1 ) gecko / 20091014 firefox / 3. 6b1 gtb5**
> 
> We can find the 5 most recent requests with HTTP status code 200 by reading the provided log file excerpt. Each line represents a single request. We can find the requests with status code 200 by looking for the string "status code : 200". We can then find the most recent requests by looking at the "timestamp" field, which indicates the date and time of the request. The requests with the most recent timestamps are the 5 most recent requests with HTTP status code 200. 


In [97]:
summarize_collection(chroma_collection)
RAG_LLM.history.clear()
while True:
  question = input("Please enter your question, or type 'bye' to exit: ")
  if question == "bye":
    print("Thank you for using the service. Goodbye!")
    break
  else:
    generateAnswer(RAG_LLM, chroma_collection, question)




Summarizing the collection...
	 Collection name: Lines
	 Number of document chunks in collection: 390
	 Distinct 'document' metadata in the collection:
	  duzenli_log_dosyasi_400.csv
Collection summarization completed.
Please enter your question, or type 'bye' to exit: bye
Thank you for using the service. Goodbye!


In [98]:
%pip install gradio
import gradio as gr


In [ ]:
RAG_LLM.history.clear()

# Replace with your actual function (assuming it generates an answer)
def generateAnswerInterFace(question):
    return generateAnswer(RAG_LLM, chroma_collection, question)

# Function to generate the info text
def get_info_text():
    return "INFO: " + summarize_collection(chroma_collection)
    # Assuming summarize_collection returns a string

# Use gr.Blocks instead of gr.Interface
with gr.Blocks() as demo:
    # Define interface components
    query_txt = gr.Textbox(label="Enter your question here:", placeholder="Type your question")
    answer_txt = gr.Textbox(label="Answer:", placeholder="Answer will be displayed here")

    # Create a button to trigger the prediction
    btn = gr.Button("Generate Answer")

    # Define the prediction function (order changed for button placement)
    def predict(question):
        answer = generateAnswerInterFace(question)
        return answer

    info_txt = gr.Textbox(get_info_text(), label="Info")  # Add info textbox after button

    # Connect button click to prediction function
    btn.click(predict, inputs=query_txt, outputs=answer_txt)

# Launch the interface
demo.launch(debug=True)


Summarizing the collection...
	 Collection name: Lines
	 Number of document chunks in collection: 390
	 Distinct 'document' metadata in the collection:
	  duzenli_log_dosyasi_400.csv
Collection summarization completed.
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://6d443aa289ea6dbb56.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


> The last request was "get / blog / geekery / headless - wrapper - for - ephemeral - xservers. html http / 1. 1" and the response status code was 200.
> 
> Here is the breakdown:
> 
> 1. The question asks for the last request and the response status code.
> 2. The provided text excerpts contain information about various requests and their corresponding status codes, timestamps, IP addresses, etc.
> 3. The last request mentioned in the text excerpts is "get / blog / geekery / headless - wrapper - for - ephemeral - xservers. html http / 1. 1".
> 4. This request has a status code of 200, as indicated in the text excerpt.
> 
> Therefore, the last request was "get / blog / geekery / headless - wrapper - for - ephemeral - xservers. html http / 1. 1" and the response status code was 200. 


> The last request was "get / images / web / 2009 / banner. png http / 1. 1" and the response status code was 200. 
> 
> Here's the breakdown:
> 
> 1. The question asks for the last request and the response status code.
> 2. We need to find the last entry in the provided text excerpts.
> 3. The final excerpt shows the following information:
>     * `ip : 125. 213. 206. 180`
>     * `timestamp : 12 / may / 2022 : 01 : 28 : 57 + 0000`
>     * `request : get / images / web / 2009 / banner. png http / 1. 1`
>     * `status code : 200`
>     * `response size : 52315`
>     * `referer : http : / / www. semicomplete. com / projects / xdotool /`
>     * `user agent : mozilla / 5. 0 ( windows nt 6. 3 ; wow64 ; trident / 7. 0 ; tnjb ; rv : 11. 0 ) like gecko`
> 4. Therefore, the last request was "get / images / web / 2009 / banner. png http / 1. 1" and the response status code was 200. 


> Son istek "get / images / jordan - 80. png http / 1. 0" idi ve yanıt durumu kodu 200'dü.
> 
> İşte adım adım açıklama:
> 
> 1. Soru, son isteğin ve yanıt durum kodunun ne olduğunu soruyor.
> 2. Sağlanan metin alıntıları, çeşitli istekler ve bunlara karşılık gelen durum kodları, zaman damgaları, IP adresleri vb. hakkında bilgi içeriyor.
> 3. Metin alıntıları içinde geçen son istek "get / images / jordan - 80. png http / 1. 0".
> 4. Bu isteğin durum kodu, metin alıntısında belirtildiği gibi 200'dür.
> 
> Bu nedenle son istek "get / images / jordan - 80. png http / 1. 0" idi ve yanıt durumu kodu 200'dü.


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 288, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1931, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1516, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

> 12 Mayıs 2022 tarihinde saat 01:24:00 ile 01:25:00 arasında sadece bir istek var. Bu istek `ip : 198. 27. 88. 92, timestamp : 12 / may / 2022 : 01 : 29 : 22 + 0000, request : get / blog / tags / x11 http / 1. 0, status code : 200, response size : 32742, referer : http : / / www. semicomplete. com / blog / tags / x11, user agent : mozilla / 5. 0 ( windows nt 6. 1 ; rv : 12. 0 ) gecko / 20100101 firefox / 12. 0` olarak kaydedilmiş. 


> Verilen metinlerde belirli bir yönlendirme URL'si (referrer URL) ile ilgili bir bilgi bulunmuyor.  Tüm isteklerin yönlendirme URL'si "-" olarak belirtilmiş. Dolayısıyla, belirli bir yönlendirme URL'sine sahip istekler hakkında bilgi veremem. 


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 288, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1931, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1516, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  File "/usr/local/lib/python3.10/dist-packages/anyio/to_thread.py", line 33, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 877, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.10/dist-packages/anyio/_backends/_asyncio.py", line 8

> 12 Mayıs 2022 tarihinde saat 01:24:00 ile 01:25:00 arasında sadece bir istek kaydedilmiş:
> 
> * **IP:** 14. 49. 42. 25
> * **Zaman Damgası:** 12 / may / 2022 : 01 : 25 : 25 + 0000
> * **İstek:** get / articles / week - of - unix - tools / http / 1. 1
> * **Durum Kodu:** 200
> * **Yanıt Boyutu:** 9313
> * **Yönlendirme URL'si:** -
> * **Kullanıcı Ajanı:** mozilla / 5. 0 ( windows ; u ; windows nt 6. 1 ; en - us ; rv : 1. 9. 2b1 ) gecko / 20091014 firefox / 3. 6b1 gtb5
> 
> Bu istek, `articles/week-of-unix-tools` yoluna yapılan bir GET isteği ve 200 durum kodu ile başarılı bir şekilde tamamlanmıştır. 


DENEME SORUSU1:What are all requests between 01:24:00 and 01:25:00 on May 12, 2022?
bu soruya türkçe yanıt oluştur

CEVABI: 12 Mayıs 2022 tarihinde saat 01:24:00 ile 01:25:00 arasında sadece bir istek kaydedilmiş:

* **IP:** 14. 49. 42. 25
* **Zaman Damgası:** 12 / may / 2022 : 01 : 25 : 25 + 0000
* **İstek:** get / articles / week - of - unix - tools / http / 1. 1
* **Durum Kodu:** 200
* **Yanıt Boyutu:** 9313
* **Yönlendirme URL'si:** -
* **Kullanıcı Ajanı:** mozilla / 5. 0 ( windows ; u ; windows nt 6. 1 ; en - us ; rv : 1. 9. 2b1 ) gecko / 20091014 firefox / 3. 6b1 gtb5

Bu istek, `articles/week-of-unix-tools` yoluna yapılan bir GET isteği ve 200 durum kodu ile başarılı bir şekilde tamamlanmıştır.